<a href="https://colab.research.google.com/github/DrVenkataRajeshKumar/S4/blob/master/trail2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [0]:
# Train Phase transformations
train_transforms = transforms.Compose([
                                       transforms.RandomRotation((-7.0, 7.0), fill=(1,)),
                                       transforms.ToTensor(),
                                       transforms.Normalize((0.1307,), (0.3081,)) 
                                       ])

# Test Phase transformations
test_transforms = transforms.Compose([
                                       transforms.ToTensor(),
                                       transforms.Normalize((0.1307,), (0.3081,))
                                       ])


In [0]:
train = datasets.MNIST('./data', train=True, download=True, transform=train_transforms)
test = datasets.MNIST('./data', train=False, download=True, transform=test_transforms)

In [38]:
SEED = 1

# CUDA?
cuda = torch.cuda.is_available()
print("CUDA Available?", cuda)

# For reproducibility
torch.manual_seed(SEED)

if cuda:
    torch.cuda.manual_seed(SEED)

# dataloader arguments - something you'll fetch these from cmdprmt
dataloader_args = dict(shuffle=True, batch_size=128, num_workers=4, pin_memory=True) if cuda else dict(shuffle=True, batch_size=64)

# train dataloader
train_loader = torch.utils.data.DataLoader(train, **dataloader_args)

# test dataloader
test_loader = torch.utils.data.DataLoader(test, **dataloader_args)

CUDA Available? False


In [0]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 8, 3, padding=0, bias=False)
        self.bn1   = nn.BatchNorm2d(8)
        self.drop1 = nn.Dropout2d(0.05)
        self.conv2 = nn.Conv2d(8, 8, 3, padding=0, bias=False)
        self.bn2   = nn.BatchNorm2d(8)
        self.drop2 = nn.Dropout2d(0.05)
        self.pool1 = nn.MaxPool2d(2, 2)
        self.conv3 = nn.Conv2d(8, 16, 3, padding=0, bias=False)
        self.bn3   = nn.BatchNorm2d(16)
        self.drop3 = nn.Dropout2d(0.05)
        self.conv4 = nn.Conv2d(16, 32, 3, padding=1, bias=False)
        self.bn4   = nn.BatchNorm2d(32)
        self.drop4 = nn.Dropout2d(0.05)
        self.gap   = nn.AvgPool2d(3)
        self.conv5 = nn.Conv2d(32, 10, 3, bias=False)
    
        

    def forward(self, x):
        x = F.relu(self.drop1(self.bn1(self.conv1(x))))
        x = self.pool1(F.relu(self.drop2(self.bn2(self.conv2(x)))))
        x = F.relu(self.drop3(self.bn3(self.conv3(x))))
        x = F.relu(self.drop4(self.bn4(self.conv4(x))))
        x = self.gap(x)
        x = self.conv5(x)
        x = x.view(-1, 10)
        return F.log_softmax(x)

In [40]:
!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
print(device)
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

cpu
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 8, 26, 26]              72
       BatchNorm2d-2            [-1, 8, 26, 26]              16
         Dropout2d-3            [-1, 8, 26, 26]               0
            Conv2d-4            [-1, 8, 24, 24]             576
       BatchNorm2d-5            [-1, 8, 24, 24]              16
         Dropout2d-6            [-1, 8, 24, 24]               0
         MaxPool2d-7            [-1, 8, 12, 12]               0
            Conv2d-8           [-1, 16, 10, 10]           1,152
       BatchNorm2d-9           [-1, 16, 10, 10]              32
        Dropout2d-10           [-1, 16, 10, 10]               0
           Conv2d-11           [-1, 32, 10, 10]           4,608
      BatchNorm2d-12           [-1, 32, 10, 10]              64
        Dropout2d-13           [-1, 32, 10, 10]               0
        AvgPool2d-14             [-

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:30: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


In [0]:
from tqdm import tqdm

train_losses = []
test_losses = []
train_acc = []
test_acc = []

def train(model, device, train_loader, optimizer, epoch):
  model.train()
  pbar = tqdm(train_loader)
  correct = 0
  processed = 0
  for batch_idx, (data, target) in enumerate(pbar):
    # get samples
    data, target = data.to(device), target.to(device)

    # Init
    optimizer.zero_grad()
    # In PyTorch, we need to set the gradients to zero before starting to do backpropragation because PyTorch accumulates the gradients on subsequent backward passes. 
    # Because of this, when you start your training loop, ideally you should zero out the gradients so that you do the parameter update correctly.

    # Predict
    y_pred = model(data)

    # Calculate loss
    loss = F.nll_loss(y_pred, target)
    train_losses.append(loss)

    # Backpropagation
    loss.backward()
    optimizer.step()

    # Update pbar-tqdm
    
    pred = y_pred.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
    correct += pred.eq(target.view_as(pred)).sum().item()
    processed += len(data)

    pbar.set_description(desc= f'Loss={loss.item()} Batch_id={batch_idx} Accuracy={100*correct/processed:0.2f}')
    train_acc.append(100*correct/processed)

def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    test_losses.append(test_loss)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))
    
    test_acc.append(100. * correct / len(test_loader.dataset))

In [42]:
model =  Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
EPOCHS = 16
for epoch in range(EPOCHS):
    print("EPOCH:", epoch)
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

  0%|          | 0/938 [00:00<?, ?it/s]/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:30: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
Loss=2.3122355937957764 Batch_id=1 Accuracy=13.28:   0%|          | 2/938 [00:00<01:07, 13.77it/s]

EPOCH: 0


Loss=0.10568507015705109 Batch_id=937 Accuracy=92.89: 100%|██████████| 938/938 [00:59<00:00, 16.30it/s]
Loss=0.12154708802700043 Batch_id=1 Accuracy=97.66:   0%|          | 2/938 [00:00<01:05, 14.37it/s]


Test set: Average loss: 0.0623, Accuracy: 9810/10000 (98.10%)

EPOCH: 1


Loss=0.03705036640167236 Batch_id=937 Accuracy=97.31: 100%|██████████| 938/938 [00:59<00:00, 15.55it/s]
Loss=0.03545954078435898 Batch_id=1 Accuracy=100.00:   0%|          | 2/938 [00:00<01:07, 13.80it/s]

...


Test set: Average loss: 0.0209, Accuracy: 9931/10000 (99.31%)

EPOCH: 14


Loss=0.00996200367808342 Batch_id=937 Accuracy=98.81: 100%|██████████| 938/938 [00:59<00:00, 16.31it/s]
Loss=0.016766613349318504 Batch_id=2 Accuracy=97.40:   0%|          | 2/938 [00:00<01:00, 15.43it/s]


Test set: Average loss: 0.0213, Accuracy: 9931/10000 (99.31%)

EPOCH: 15


Loss=0.00953737273812294 Batch_id=937 Accuracy=98.88: 100%|██████████| 938/938 [01:00<00:00, 16.30it/s]



Test set: Average loss: 0.0209, Accuracy: 9939/10000 (99.39%)

